In [1]:
import pandas as pd
import numpy as np
from Classes import Sortino
import joblib

In [2]:
prices_df = pd.read_excel("daily_dbs/daily_dbs_sp50&nasdaq.xlsx")
market_features_df = pd.read_excel("daily_dbs/daily_market_features.xlsx")
rfr_df = pd.read_csv('daily_dbs/risk_free_rate.csv')

selección aleatoria de activos

In [3]:
returns_df = pd.read_csv("daily_dbs/returns_df.csv")

llamar al clasificador

In [82]:
tickers = returns_df.columns.tolist()
selected_assets = np.random.choice(tickers, 20, replace=False).tolist()
selected_assets

pkl_path = "trained_models/kmeans_model.pkl"
market_caps_path = "market_caps_kmeans.csv"

kmeans, cluster_map, _, _ = joblib.load(pkl_path)
print("Portafolio temporal:", selected_assets)

df = pd.read_csv(market_caps_path)
df = df.dropna(subset=["Market Cap"])
df = df.set_index("Ticker")
market_caps = df["Market Cap"].to_dict()

# pesos equitativos por default
weights = [1/len(selected_assets)] * len(selected_assets)
cap = sum([market_caps.get(t, 0) * w for t, w in zip(selected_assets, weights)])
log_cap = np.log1p(cap)
cluster = kmeans.predict([[log_cap]])[0]
etiqueta = cluster_map[cluster]
desc = {-1: 'Low Cap', 0: 'Mid Cap', 1: 'High Cap'}

print("Etiqueta asignada:", etiqueta)

Portafolio temporal: ['BLDR', 'AWK', 'MMC', 'SWKS', 'AIZ', 'ULTA', 'CSGP', 'CPRT', 'COR', 'ALB', 'REGN', 'BA', 'MGM', 'SPGI', 'TMUS', 'CSX', 'ALGN', 'D', 'VMC', 'MCD']
Etiqueta asignada: -1


llamamos a la clase de sortino que hace toda la chamba :V

In [ ]:
rfr_path = 'daily_dbs/risk_free_rate.csv'
sortino = Sortino(returns_df, rfr_path, selected_assets)


sortino.generate_multiple_weights(num_combinations=100)
sortino_returns = sortino.calculate_portfolio_returns()
#sortino.calculate_sortino_ratio()
sortino.calculate_sharpe_ratio()

sharpe_dataset = sortino.create_portfolio_dataset()
sharpe_dataset.head()

database_daily = pd.merge(sharpe_dataset, market_features_df, on="Date", how="inner")
database_daily.to_csv("daily_dbs/sharpe_daily_LowCap.csv")

Generadas 50 combinaciones de pesos para cada fecha.
Rendimientos del portafolio calculados para cada combinación de pesos.
Ratios de Sharpe calculados para cada combinación de pesos.


,Date,Weight_PWR,Weight_IBM,Weight_GL,Weight_ON,Weight_WAB,Weight_DOC,Weight_XEL,Weight_TECH,Weight_AAPL,...,Weight_STT,Weight_TEL,Weight_AVY,Weight_PFE,Weight_ZBH,Weight_DHI,Weight_MLM,Weight_TMUS,Portfolio_Returns,Sharpe_Ratio
0,2010-01-05,1.438475,2.049315,6.314220,7.262369,7.862904,8.602548,3.485468,4.250166,2.302822,...,2.463152,4.186294,7.445550,6.240213,7.886575,7.978566,1.675598,5.135198,0.003117,1.356262
1,2010-01-05,1.092717,1.842622,7.423569,1.855472,8.827137,8.112234,5.502423,6.704620,1.553476,...,5.479378,3.244236,5.353535,6.675871,0.396798,6.748110,6.628214,9.466458,-0.000025,-0.079608
2,2010-01-05,9.110814,3.633182,7.043180,0.689633,4.588166,7.549764,6.017841,0.952999,3.086019,...,4.992175,0.287465,8.417127,2.334142,2.631780,8.580843,0.695888,9.046722,0.003930,1.727721
3,2010-01-05,3.195893,8.488847,11.149827,3.598420,2.177485,5.379604,1.708935,10.256404,0.934720,...,6.038790,0.689136,0.230667,1.745844,1.399227,4.439765,8.759926,8.310939,-0.000289,-0.200582
4,2010-01-05,2.024997,0.067142,8.108239,0.215995,0.244351,0.970228,7.802397,5.896684,0.544479,...,8.194048,1.490469,6.000647,0.964251,12.748774,10.714492,3.174438,9.994068,0.007451,3.337210


database para el backtesting

In [3]:
database_bt = pd.merge(prices_df, market_features_df, on="Date", how="inner")
database_bt.head()

,Date,CTAS,RTX,WEC,MAA,AES,FAST,ANSS,ED,EQIX,...,COO,Treasury Bond 10Y,Treasury Bond 3M,WTI Index,Yield Curve Spread (10Y - 2Y),USD Index,MOM,VVIX,SMB,HML
0,2010-01-04,5.795176,31.163885,14.957690,27.001556,9.061471,3.075539,44.639999,24.718466,82.410835,...,9.377275,3.85,0.08,81.52,2.76,77.53,0.58,20.04,-0.14,0.21
1,2010-01-05,5.786392,30.698355,14.762187,26.979151,8.968672,3.139202,43.639999,24.397079,81.643608,...,9.486571,3.77,0.07,81.74,2.76,77.62,0.62,19.35,0.09,0.05
2,2010-01-06,5.801763,30.537390,14.912576,27.068806,8.875872,3.346286,43.349998,24.658539,82.388283,...,9.481605,3.85,0.06,83.12,2.84,77.49,-0.05,19.16,-0.33,0.83
3,2010-01-07,5.852271,30.667902,14.882504,27.197664,8.882501,3.403363,43.459999,24.571383,80.703362,...,9.580969,3.85,0.05,82.60,2.82,77.91,-0.87,19.06,-0.10,0.09
4,2010-01-08,5.909366,30.728806,14.969725,27.018373,9.134392,3.437755,43.759998,24.647646,80.312218,...,9.426954,3.83,0.05,82.74,2.87,77.47,0.20,18.13,0.18,-0.28


In [ ]:
database_bt.to_csv("daily_dbs/sortino_tuning.csv")

In [5]:
database_bt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3771 entries, 0 to 3770
Columns: 435 entries, Date to HML
dtypes: datetime64[ns](1), float64(434)
memory usage: 12.5 MB
